In [1]:
import pyspark
import pyspark.sql.functions as F
import shutil
from batcave.scrape_and_clean import get_item_image, row_to_html_card
from pyspark.sql.types import StringType, IntegerType

In [2]:
spark = (pyspark.sql.SparkSession.builder
    .master("local")
    .getOrCreate())

In [3]:
all_reviews = spark.read.json('data/all_reviews_fixed_titles.json')

### Getting thumbnail images for web interface
Lucky for me, the Amazon review data I have been working with has links to the tumbnail images for each product.  I wrote a function using ```urllib``` to download those images locally for use later. I downloaded all of the top 1,000 most frequently rated movies (as these will be my initial basis I will be drawing from) and all of the covers for comic books to give it the full scope of options to pull from when recommending. 

In [4]:
top_movie_url_query = """
SELECT
    DISTINCT asin
,   title
,   imUrl
,   item_id
,   count
FROM
    all_reviews
WHERE 
    item_id LIKE "%44"
ORDER BY 
    count DESC
LIMIT 1000
"""

In [5]:
all_reviews.createOrReplaceTempView('all_reviews')
movie_image_urls = spark.sql(top_movie_url_query).toPandas()

In [6]:
movie_missing_urls = movie_image_urls[movie_image_urls['imUrl'].isna()]
movie_active_urls = movie_image_urls[movie_image_urls['imUrl'].isna() == False]

In [7]:
movie_image_urls.head()

,asin,title,imUrl,item_id,count
0,B001KVZ6HK,Marvel's: The Avengers,http://ecx.images-amazon.com/images/I/617wvf8S...,234044,170
1,B008JFUPFI,Man of Steel,http://ecx.images-amazon.com/images/I/51vg6U6f...,579744,163
2,B00005JPY0,The Dark Knight [Theatrical Release],http://ecx.images-amazon.com/images/I/51pGxSkk...,1900344,141
3,B009934S5M,Star Trek Into Darkness,http://ecx.images-amazon.com/images/I/51K7eJ6I...,531144,140
4,B0001VL0K2,The Lord of the Rings: The Motion Picture Tril...,http://ecx.images-amazon.com/images/I/51z4rwl-...,902244,133


In [ ]:
get_item_image(comic_active_urls, 'images/movies/')

In [46]:
top_comic_url_query = f"""
SELECT
    DISTINCT asin
,   imUrl
,   count
,   item_id
FROM
    all_reviews
WHERE 
    item_id LIKE "%22"
ORDER BY 
    count DESC
"""

In [49]:
all_reviews.createOrReplaceTempView('all_reviews')
comic_image_urls = spark.sql(top_comic_url_query)

In [50]:
comic_image_urls.repartition(1).write.json("combining")

In [12]:
comic_missing_urls = comic_image_urls[comic_image_urls['imUrl'].isna()]
comic_active_urls = comic_image_urls[comic_image_urls['imUrl'].isna() == False]

In [13]:
comic_active_urls.head()

,asin,imUrl,count,item_id
0,1607060760,http://ecx.images-amazon.com/images/I/51m-0BhI...,314,521522
1,1607065967,http://ecx.images-amazon.com/images/I/51CnU-l0...,193,301322
2,1401220347,http://ecx.images-amazon.com/images/I/81tnMIDL...,137,427622
3,1401235417,http://ecx.images-amazon.com/images/I/81%2B3jj...,127,261522
4,1607066017,http://ecx.images-amazon.com/images/I/515jeiad...,120,217522


In [14]:
get_item_image(comic_active_urls, 'images/comics/')

Since I just needed to manually find covers for these last two, I searched for them by ASIN on Amazon and downloaded to the same directory:

In [10]:
comic_missing_urls

,asin,imUrl,count
245,0785142592,None,18
1014,0785148744,None,7


### Selecting movies to use
My goal for the web app interface is to offer a diverse selection of movies for users to select to help gather a recommendation.  To best solve this, I need to manually review this set of most reviewed movies and create a subsection to give a good mix of genres. I want to limit the amount of movies that are from comic source material as to avoid the really obvious connections (eg. You love Iron Man and get a recommendation for Iron Man, far too obvious a leap). 

* Remove any titles I don't want to feature
* Do further name cleaning
* Figure out way to get featured images for both
* Hulk vs?, Iron Man: The Art of Iron Man 2
* Bad name features still in place: (DVD), (Widescreen Edition) - anything in parenthesis! [Blu-ray]

After review, I chose these 60 movies to make up my application, ranging in genre and age, but all not based on any preceding comic book material:

In [8]:
movies = [22, 5, 30, 35, 39, 48, 57, 58, 74, 78, 79, 80, 90, 93, 95, 101, 105,
106, 112, 120, 127, 140, 158, 161, 162, 165, 190, 194, 203, 207, 223,
234,235, 241, 252,268, 270, 278, 287, 312, 323, 325, 336, 337, 348, 
350, 351, 352, 365, 370, 392, 400, 415, 417, 429, 431, 445, 448, 463, 
476]

In [9]:
len(movies)

60

In [22]:
# Index does change with every load, so these results might not show up the same
movie_image_urls.loc[movies].sort_values('asin')

,asin,title,imUrl,item_id,count
448,0767805267,Fifth Element,http://ecx.images-amazon.com/images/I/513N2SKX...,605744,25
203,0767825411,Ghostbusters,http://ecx.images-amazon.com/images/I/51YW4X17...,1857744,37
323,0767849493,Dogma,http://ecx.images-amazon.com/images/I/51NJ5M9N...,1293544,30
445,0780623134,The Shawshank Redemption,http://ecx.images-amazon.com/images/I/51YPVEBE...,1047044,26
101,0783216084,Jaws,http://ecx.images-amazon.com/images/I/514JQY1K...,1270644,49
348,0783222734,Jurassic Park,http://ecx.images-amazon.com/images/I/51MNFQR1...,1843244,29
400,0783225849,Psycho,http://ecx.images-amazon.com/images/I/515ZHBJZ...,2262644,27
112,0783241038,Galaxy Quest,http://ecx.images-amazon.com/images/I/51K4TAW5...,2465444,48
140,0788828126,Snow White and the Seven Dwarfs - Platinum Edi...,http://ecx.images-amazon.com/images/I/51F7cloW...,1004644,43
57,0788882988,No Country for Old Men,http://ecx.images-amazon.com/images/I/51pWB3JZ...,832744,62


In [24]:
def move(src, dest):
    shutil.move(src, dest)

In [32]:
# Adjust to match directory you're working in
for index, item in movie_image_urls.loc[movies].iterrows():
    try:
        file = item['asin'] + item['imUrl'][-4:]
        src =  '~/Documents/intocomics/images/movies/' + file
        dest = '~/Documents/intocomics/images/final_movies/' + file
        move(src, dest)
    except:
        continue

### Web design helper function
This will give me 15 nice rows of content for my webpage. In a different format, using something like this would be best to link a database to the data and have it populate my page.  Unfortunately, I don't have the time for that level of web development, but I can write a function to make the individual card images I need with Python! The funciton used below takes in the dataframe and returns a text document with all the html needed:

In [10]:
with open('all_movie_cards.html', 'w+') as f:
    for movie in movies:
        f.write(row_to_html_card(movie_image_urls.loc[movie]))